In [79]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 
from os import walk
import json 

In [80]:
def read_json(file_name):
    with open(file_name) as f:
      data = json.load(f)
    return data

In [44]:
b =  [1,   2,   3,  4,   5,    6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
bstress = [22, 10, 19, 17, 23, 14, 9, 18, 23, 10, 8, 13, 12, 20, 21, 13, 0, 28, 13, 7, 18, 13, 15]
g =  [1,    2,   3,  4,  5,  6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17,]
gstress =  [11, 13, 25, 7, 28, 14, 8, 13, 10, 12, 11, 16, 20, 25, 16, 31, 9]

In [55]:
df = pd.read_csv('../table.csv')

In [56]:
df.columns

Index(['Exp_ID', 'exp_time', 'trais no.', 'note', 'signal_condition', 'gender',
       'age', 'race', 'handiness', 'sleep condition', 'Q11', 'Q12', 'Q13',
       'Q13_r', 'Q14', 'ID', 'ID_boy', 'ID_girl', 'Raw_within_RF',
       'Raw_within_logistic', 'within_classification',
       'within_classification_logistic', 'Raw_within_netVSneg_RF',
       'Raw_within_netVSneg_logistic', 'Within_netVSneg_RF',
       'Within_netVSneg_logistic', 'within_Feature_asymmtry_RF',
       'within_Feature_Less_RF', 'within_Feature_Less_Logistic', 'stress',
       'stressGroup', '42set_ID', 'between_classification_all',
       'Between_Feature_asymmtry', 'Between_Feature_Less',
       'raw_betweenNeutral2Negative', 'raw_StressGroupbetweenNeutral2Negative',
       'raw_between_stressGroup', 'between_stressGroup',
       'Between_Stress2groups_RF', 'Between_Stress2groups_log',
       'Between_GenderGroup', 'Between_RandomGroup',
       'between_stressGroup_netural_vs_negative', '39set_ID',
       'between_

In [82]:
folder = '../results/sleep_condition/'

In [119]:
_, _, filenames = next(walk(folder))

In [120]:
filenames.sort()

In [121]:
filenames

['bad.json', 'good.json']

In [122]:
bad = read_json(folder+filenames[0])['avg_acc']
good = read_json(folder+filenames[1])['avg_acc']

In [123]:
id_bad = [2,3,4,7,9,12,14,18,27,29,32] 
id_good = [0,1,5,17,20,21,23,24,28,37,38]

In [124]:
for i in range(len(bad)):
    bad[i] = [id_bad[i],bad[i]]
for i in range(len(good)):
    good[i] = [id_good[i],good[i]]
print(len(bad),len(good))
overall = [] 
bad.extend(good)
overall = bad
len(overall)

11 11


22

In [102]:
overall.sort(key=lambda x:x[0])
good.sort(key=lambda x:x[0])

In [104]:
for i in range(len(overall)):
    overall[i] = overall[i][1]

In [109]:
for _ in range(20):
    overall.append(-1)
df['Between_sleep_RF'] = overall[:52]
df.to_csv('../table.csv')